# Quickstart: assess impacts of a user journey

This notebook provides an example scenario that you can use to get familiar with the Pyhton API of efootprint.

It builds a simulation scenario, step by step, made of:
- multiple resources involved (serveurs with auto-scaling settings, storage, service and network)
- A user journey involving 2 steps (Streaming, Upload)
- a usage pattern

## Import the packages

⚠ If this steps fails, remember to execute it _inside_ your python virtual environement. (You may initialize it with `poetry install`).

In [93]:
from efootprint.constants.sources import SourceValue, Sources, SourceObject
from efootprint.core.usage.user_journey import UserJourney, UserJourneyStep
from efootprint.core.hardware.servers.autoscaling import Autoscaling
from efootprint.core.hardware.storage import Storage
from efootprint.core.service import Service
from efootprint.core.hardware.device_population import DevicePopulation
from efootprint.core.usage.usage_pattern import UsagePattern
from efootprint.core.hardware.network import Network
from efootprint.core.system import System
from efootprint.constants.countries import Countries
from efootprint.constants.units import u
from efootprint.utils.calculus_graph import build_calculus_graph
from efootprint.utils.object_relationships_graphs import build_object_relationships_graph, \
    USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE
from efootprint.builders.hardware.devices_defaults import default_laptop




## Define the resources

In [94]:
server = Autoscaling(
    "Autoscaling server",
    carbon_footprint_fabrication=SourceValue(600 * u.kg, Sources.BASE_ADEME_V19),
    power=SourceValue(300 * u.W, Sources.HYPOTHESIS),
    lifespan=SourceValue(6 * u.year, Sources.HYPOTHESIS),
    idle_power=SourceValue(50 * u.W, Sources.HYPOTHESIS),
    ram=SourceValue(128 * u.GB, Sources.HYPOTHESIS),
    nb_of_cpus=SourceValue(24 * u.core, Sources.HYPOTHESIS),
    power_usage_effectiveness=SourceValue(1.2 * u.dimensionless, Sources.HYPOTHESIS),
    average_carbon_intensity=SourceValue(100 * u.g / u.kWh, Sources.HYPOTHESIS),
    server_utilization_rate=SourceValue(0.9 * u.dimensionless, Sources.HYPOTHESIS))

In [95]:
storage = Storage(
    "SSD storage",
    carbon_footprint_fabrication=SourceValue(160 * u.kg, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    power=SourceValue(1.3 * u.W, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    lifespan=SourceValue(6 * u.years, Sources.HYPOTHESIS),
    idle_power=SourceValue(0 * u.W, Sources.HYPOTHESIS),
    storage_capacity=SourceValue(1 * u.TB, Sources.STORAGE_EMBODIED_CARBON_STUDY),
    power_usage_effectiveness=SourceValue(1.2 * u.dimensionless, Sources.HYPOTHESIS),
    average_carbon_intensity=SourceValue(100 * u.g / u.kWh),
    data_replication_factor=SourceValue(3 * u.dimensionless, Sources.HYPOTHESIS))

In [96]:
service = Service(
    "Youtube",
    server=server,
    storage=storage,
    base_ram_consumption=SourceValue(300 * u.MB, Sources.HYPOTHESIS),
    base_cpu_consumption=SourceValue(2 * u.core, Sources.HYPOTHESIS))

## Define the user journey

The journey spans multiple steps:
- Streaming
- Upload

### Define the First step (Streaming)

In [97]:
streaming_step = UserJourneyStep(
    "20 min streaming on Youtube",
    service=service,
    data_upload=SourceValue(50 * u.kB / u.uj, Sources.USER_INPUT),
    data_download=SourceValue((2.5 / 3) * u.GB / u.uj, Sources.USER_INPUT),
    user_time_spent=SourceValue(20 * u.min / u.uj, Sources.USER_INPUT),
    request_duration=SourceValue(4 * u.min, Sources.HYPOTHESIS),
    cpu_needed=SourceValue(1 * u.core / u.uj, Sources.HYPOTHESIS),
    ram_needed=SourceValue(50 * u.MB / u.uj, Sources.HYPOTHESIS))

2024-01-19 17:13:04,739 - INFO - Computing calculated attributes for UserJourneyStep 20 min streaming on Youtube


### Define the second step (Upload)

In [98]:
upload_step = UserJourneyStep(
    "0.4s of upload",
    service=service,
    data_upload=SourceValue(300 * u.kB / u.uj, Sources.USER_INPUT),
    data_download=SourceValue(0 * u.GB / u.uj, Sources.USER_INPUT),
    user_time_spent=SourceValue(0.4 * u.s / u.uj, Sources.USER_INPUT),
    request_duration=SourceValue(0.4 * u.s, Sources.HYPOTHESIS),
    cpu_needed=SourceValue(1 * u.core / u.uj, Sources.HYPOTHESIS),
    ram_needed=SourceValue(500 * u.MB / u.uj, Sources.HYPOTHESIS)
)

2024-01-19 17:13:04,843 - INFO - Computing calculated attributes for UserJourneyStep 0.4s of upload


### Assemble the entire journey from the steps defined above

In [99]:
user_journey = UserJourney("Mean Youtube user journey", uj_steps=[streaming_step, upload_step])

2024-01-19 17:13:04,904 - INFO - Computing calculated attributes for UserJourney Mean Youtube user journey


### Explicit population, network consumption and global usage pattern

In [100]:


device_population = DevicePopulation(
    "French Youtube users on laptop",
    nb_devices=SourceValue(4e7 * 0.3 * u.user, Sources.USER_INPUT),
    country=Countries.FRANCE,
    devices=[default_laptop()])

network = Network(
        "WIFI network",
        bandwidth_energy_intensity=SourceValue(0.05 * u("kWh/GB"), Sources.TRAFICOM_STUDY))

usage_pattern = UsagePattern(
    "Daily Youtube usage",
    user_journey=user_journey,
    device_population=device_population,
    network=network,
    user_journey_freq_per_user=SourceValue(365 * u.user_journey / (u.user * u.year), Sources.USER_INPUT),
    time_intervals=SourceObject([[7, 12], [17, 23]]))

system = System("system 1", usage_patterns=[usage_pattern])



2024-01-19 17:13:04,949 - INFO - Computing calculated attributes for DevicePopulation French Youtube users on laptop
2024-01-19 17:13:04,960 - INFO - Computing calculated attributes for UsagePattern Daily Youtube usage
2024-01-19 17:13:04,964 - INFO - Computing calculated attributes for DevicePopulation French Youtube users on laptop
2024-01-19 17:13:04,980 - INFO - Computing calculated attributes for Service Youtube
2024-01-19 17:13:05,021 - INFO - Computing calculated attributes for Network WIFI network
2024-01-19 17:13:05,038 - INFO - Computing calculated attributes for Autoscaling Autoscaling server
2024-01-19 17:13:05,115 - INFO - Computing calculated attributes for Storage SSD storage


### Simulate and show the results

In [101]:
print(f"Server carbon footprint is {(server.energy_footprint + server.instances_fabrication_footprint).value}")
print(f"Total system carbon footprint is {system.total_footprint().value}")


Server carbon footprint is 707604.7183333335 kilogram / year
Total system carbon footprint is 37408251.09958489 kilogram / year


## Show calculus graph

In [102]:
# current_dir = os.getcwd()
# current_dir = os.path.abspath(os.path.dirname(__file__))

calculus_graph = build_calculus_graph(device_population.instances_fabrication_footprint)

calculus_graph.show("device_population_fab_footprint_calculus_graph.html")
# calculus_graph.show(os.path.join(current_dir, "device_population_fab_footprint_calculus_graph.html"))


device_population_fab_footprint_calculus_graph.html


## Show relations graph

In [103]:

object_relationships_graph = build_object_relationships_graph(
     usage_pattern, classes_to_ignore=USAGE_PATTERN_VIEW_CLASSES_TO_IGNORE)
# object_relationships_graph.show(os.path.join(current_dir, "object_relationships_graph.html"))
object_relationships_graph.show("object_relationships_graph.html")

object_relationships_graph.html
